In [2]:
import math
import point_similarity as ps
import scipy.integrate as integrate
import pylab
import mpmath
#from scipy.misc import derivative
import plotly
from plotly import graph_objs as go
import sys
import read_arbor_reconstruction as rar
import networkx as nx
import pareto_functions as pf
from constants import *
import os
import scipy
import argparse
import pandas as pd
from scipy.optimize import root_scalar
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import fsolve

pi= math.pi
import math
def cos(t):
    return math.cos(t)
def sin(t):
    return math.sin(t)
def sqrt(t):
    return np.sqrt(t)


from scipy.optimize import fsolve

def find_multiple_roots(func, x_range, num_guesses=100, tol=1e-5):
    roots = []
    guesses = np.linspace(x_range[0], x_range[1], num_guesses)
    for guess in guesses:
        try:
            root_arr = fsolve(func, guess)
            root = root_arr[0]
            # Check if it's a valid root
            if abs(func(root)) < tol and not any(np.isclose(root, r, atol=1e-5) for r in roots):
                roots.append(root)
        except Exception as e:
            print(f"Solver failed at guess {guess}: {e}")
    return sorted(roots)

def f(x):
    return costprime(x)


## returns the coefficients of the quadratic equation involving gravity
def calc_coeff(G, x, y, p, q):
    b = ((q - y - G*(p*p - x*x))/(p-x))
    c = (q - G*p*p - b*p)

    # assert (G * x * x) + (b * x) + c == y
    # assert (G * p * p) + (b * p) + c == q

    return b, c

## checks if a line has a positive slope given two points
def positive_slope(x0, y0, x1, y1):
    if x0 == x1 or y0 == y1:
        return False
    slope = (y1 - y0)/(x1 - x0)
    if slope >= 0:
        return True
    return False

## simply just a distance function
def length_func(x0, y0, x1, y1):
    length = pylab.sqrt((x1 - x0)**2 + (y1 - y0)**2)
    return length

## a way to calculate theta based on two points
def get_theta(x0, y0, x1, y1):
    if x1 == x0 or y1 == y0:
        return 0
    theta = pylab.arctan((abs(y1 - y0))/(abs(x1 - x0)))
    return theta

## Returns the length of a curve (in this case a lateral root when gravity is not 0)
def curve_length(G, x0, y0, p, q):
    b, c = calc_coeff(G, x0, y0, p, q)
    def differential(x):
        return pylab.sqrt(1 + (2*G*x + b)**2)
    curve, tolerance = integrate.quad(differential, min(x0, p), max(x0, p))
    return curve

## Returns the distance from a point to the base of the plant
def distance_from_base(root_distance, x, y, x1, y1):
    pair1 = [x, y]
    pair2 = [x1, y1]
    distance = math.dist(pair1, pair2)
    if(x,y) == (x1, y1):
        assert distance == 0
    return distance + root_distance

## Returns the total cost along with the wiring and delay used to calculate it
def total_cost(alpha, G, root_distance, x0, y0, x1, y1, p, q):
    curve = curve_length(G, x1, y1, p, q)
    to_root = distance_from_base(root_distance, x0, y0, x1, y1)
    wiring = curve
    delay = (curve + to_root)
    cost = (alpha * wiring) +  ((1 - alpha) * delay)
    #print("curve = ", curve, "to root = " , to_root, "wiring = ", wiring, "delay = ", delay, "total cost = ", cost)
    #cost = curve + (1 - alpha) * distance_from_base(root_distance, x0, y0, x1, y1)
    return cost, wiring, delay


def find_best_cost(alpha, G, root_distance, x0, y0, x1, y1, p, q):
    l = length_func(x0, y0, x1, y1)
    theta = get_theta(x0, y0, x1, y1)
    is_positive = positive_slope(x0, y0, x1, y1)

    def costprime(t):
        A1 = G * (l * math.cos(theta)) ** 2
        B1 = -l * math.sin(theta)
        C1 = q - G * p ** 2
        D1 = -l * math.cos(theta)
        E1 = p

        def b(t): 
            return (A1 * t ** 2 + B1 * t + C1) / (D1 * t + E1)
        def bprime(t):
            return ((D1 * t + E1) * (2 * A1 * t + B1) - D1 * (A1 * t ** 2 + B1 * t + C1)) / (D1 * t + E1) ** 2

        return (bprime(t)/(2 * G)) * (
            sqrt(1 + (2 * G * p + b(t)) ** 2) - sqrt(1 + (2 * G * t * l * cos(theta) + b(t)) ** 2)
        ) + (1 - alpha) * l - sqrt(1 + (2 * G * t * l * cos(theta) + b(t)) ** 2) * l * cos(theta)

    # Find roots of costprime in [0,1]
    roots = find_multiple_roots(costprime, [0, 1], num_guesses=100)

    # Add edge cases at t=0 and t=1
    roots += [0, 1]
    best_cost = float('inf')
    best_t = None
    best_x = None
    best_y = None

    for t in roots:
        if not 0 <= t <= 1:
            continue
        new_x = x0 + t * l * math.cos(theta) if is_positive else x0 - t * l * math.cos(theta)
        new_y = y0 + t * l * math.sin(theta)

        cost, wiring, delay = total_cost(alpha, G, root_distance, x0, y0, new_x, new_y, p, q)
        if cost < best_cost:
            best_cost = cost
            best_t = t
            best_x = new_x
            best_y = new_y
            best_wiring = wiring
            best_delay = delay

    return best_cost, best_wiring, best_delay, best_t, best_x, best_y, p, q

def arbor_best_cost(fname, alpha, G, root_distance):
    arbor = rar.read_arbor_full(fname)
    main_root = []
    lat_tips = []
    line_segments = {}
    point_drawing = go.Figure()
    for node in arbor.nodes():
        if arbor.nodes[node]["label"] == ("main root") or arbor.nodes[node]["label"] == ("main root base"):
            main_root.append(node)
        if arbor.nodes[node]["label"] == ("lateral root tip"):
            lat_tips.append(node)
    for i in range(1, len(main_root)):
        line_segments[i] = main_root[i - 1], main_root[i]

    # for each lateral root tip iterate through every line segment to find lowest costing point on main root
    final = []
    for tip in lat_tips:
        curr_dist = 0
        results = []
        firstTime = True
        p = tip[0]
        q = tip[1]
        for seg in line_segments:
            x0 = line_segments[seg][0][0]
            y0 = line_segments[seg][0][1]
            x1 = line_segments[seg][1][0]
            y1 = line_segments[seg][1][1]
            if firstTime == True:
                result = find_best_cost(alpha, G, root_distance, x0, y0, x1, y1, p, q)
                curr_dist += length_func(x0, y0, x1, y1)
                firstTime = False
            else:
                result = find_best_cost(alpha, G, root_distance + curr_dist, x0, y0, x1, y1, p, q)
                curr_dist += length_func(x0, y0, x1, y1)
            results.append(result)
        final.append(min(results))

    return final


def graph_opt_lines(graph, final):
    for result in final:
        x0 = result[2]
        y0 = result[3]
        p = result[4]
        q = result[5]
        first_point = x0, y0
        second_point = p, q
        G = graph.graph['Gravity']
        b, c = calc_coeff(G, x0, y0, p, q)
        graph.add_node(first_point, pos=first_point)
        graph.add_node(second_point, pos=second_point)
        graph.add_edge(first_point, second_point, b = b, c = c)
    return graph

def get_line_segments(arbor):
    line_segments = {}
    main_root = []
    for node in arbor.nodes():
        if arbor.nodes[node]["label"] == ("main root") or arbor.nodes[node]["label"] == ("main root base"):
            main_root.append(node)
    for i in range(1, len(main_root)):
        line_segments[i] = main_root[i - 1], main_root[i]
    return line_segments


def graph_main_root(graph, line_segments):
     for seg in line_segments:
        x0 = line_segments[seg][0][0]
        y0 = line_segments[seg][0][1]
        x1 = line_segments[seg][1][0]
        y1 = line_segments[seg][1][1]
        first_point = x0, y0
        second_point = x1, y1


        graph.add_node(first_point, pos=first_point)
        graph.add_node(second_point, pos=second_point)
        graph.add_edge(first_point, second_point)


def modified_line_equation(G, main_root, lateral_tip):
    x, y = main_root
    p, q = lateral_tip
    if G == 0:
        m = (q - y) / (p - x)
        y_int = y - m * x
        return m, y_int
    else:
        b, c = calc_coeff(G, x, y, p, q)
        return b, c

def modified_calculate_distance(gravity, G, G_opt, main_root, lateral_tip):
    opt_y_coords, actual_y_coords = modified_fill_lateral_root(gravity, G, G_opt, main_root, lateral_tip)
    distances = []

    for x in range(len(opt_y_coords)):
        diff_square = (opt_y_coords[x] - actual_y_coords[x]) ** 2
        distances.append(diff_square)

    return sum(distances)


def modified_fill_lateral_root(gravity, G, G_opt, main_root, lateral_tip) :

    ## This method uses the lateral tip and main root to fill in points of the lateral root
    ## in order to perform distance calculation

    # get coefficients
    if gravity == 0:
        m, y_int = modified_line_equation(gravity, main_root, lateral_tip)
    else:
        b, c = modified_line_equation(gravity, main_root, lateral_tip)


    observed = {}
    optimal = {}
    observed, optimal = ps.create_dict(G, G_opt)
    backwards = {}

    ## reverses observed dictionary
    for node in reversed(observed):
        backwards[node] = observed[node]

    tip_found = False
    encountered_observed = []
    index = len(G.nodes) - 2

    ## loop backwards
    for node in backwards :
        if G.nodes[node]['label'] == "lateral root tip" and tip_found:
            break
        if lateral_tip == node:
            tip_found = True
           # print("tip was found")
            continue
        if tip_found == True and (list(G.nodes(data = True))[index][1]["label"] == "lateral root"):
           # print("not a tip")
            encountered_observed.append(node)
        index -= 1

    ## need to calculate points based on observed x-coordinates and line equation
    x_coords = []
    y_coords = []
    count = -1
    for point in encountered_observed:
        for coords in point:
          count += 1
          if count % 2 == 0 :
              x_coords.append(coords)
          else :
              y_coords.append(coords)
    added_nodes = []
    if gravity == 0:
        for x in x_coords:
            added_nodes.append(m * x + y_int)
    else:
        for x in x_coords:
            added_nodes.append(gravity*x*x + b*x + c)

    return added_nodes, y_coords


def calc_pareto_front(fname, amin=0, amax=1, astep=0.05, Gmin=-2, Gmax=2, Gstep=0.2, skip=None):
    G = rar.read_arbor_full(fname)
    if skip == None:
        skip = set()

    values = []
    for alpha in pylab.arange(amin, amax + astep, astep):
        for g in pylab.arange(Gmin, Gmax + Gstep, Gstep):
            if (alpha, g) in skip:
                continue
            G_opt = nx.Graph(Gravity = g)
            line_segs = get_line_segments(G)
            graph_main_root(G_opt, line_segs)
            final = arbor_best_cost(fname, alpha, g, 0) # 0 is the root distance but I figured it doesn't matter as code calculates proper root distance
            graph_opt_lines(G_opt, final)
            point_dist = 0
            #print("Calculating distances for alpha: " + str(alpha) + " and G = " + str(g))
            wiring = 0
            delay = 0
            for result in final:
                wiring += result[1]
                delay += result[2]
                main_root = result[4], result[5]
                lateral_tip = result[6], result[7]
                point_dist += modified_calculate_distance(g, G, G_opt, main_root, lateral_tip)
            wiring += main_root_distance(line_segs) # earlier, did not account for the main root when calculating wiring cost
            values.append((g, alpha, wiring, delay, point_dist))

    return values

def main_root_distance(line_segments):
    distance = 0
    for seg in line_segments:
        x0 = line_segments[seg][0][0]
        y0 = line_segments[seg][0][1]
        x1 = line_segments[seg][1][0]
        y1 = line_segments[seg][1][1]
        distance += length_func(x0, y0, x1, y1)
    return distance

def modified_conduction_delay(G):
    '''
    use a breadth-first search to compute the distance to from the root to each point

    when we encounter a visit node for the first time, we record its distance to the root
    (which is the sum of its parent's distance, plus the length of the edge from the parent
    to the current node').  We keep a running total of the total distances from the root
    to each node.
    '''
    droot = {}
    queue = []
    curr = None
    visited = set()

    root = G.graph['main root base']

    queue.append(root)
    droot[root] = 0

    delay = 0
    while len(queue) > 0:
        curr = queue.pop(0)
        # we should never visit a node twice
        assert curr not in visited
        visited.add(curr)
        # we only measure delay for the lateral root tips
        if G.nodes[curr]['label'] == 'lateral root tip':
            delay += droot[curr]
        for u in G.neighbors(curr):
            if u not in visited:
                queue.append(u)
                droot[u] = droot[curr] + G[curr][u]['length']

    # make sure we visited every node
    assert len(visited) == G.number_of_nodes()

    return delay


In [3]:
calc_pareto_front("001_1_c_day5.csv", amin=0, amax=1, astep=0.1, Gmin=-0.2, Gmax=1, Gstep=0.2, skip=None)

C:\Users\agarz\AppData\Local\Temp\ipykernel_20208\2259806315.py:41: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.

C:\Users\agarz\AppData\Local\Temp\ipykernel_20208\2259806315.py:130: RuntimeWarning:

divide by zero encountered in divide

C:\Users\agarz\AppData\Local\Temp\ipykernel_20208\2259806315.py:130: RuntimeWarning:

invalid value encountered in multiply

C:\Users\agarz\AppData\Local\Temp\ipykernel_20208\2259806315.py:130: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\agarz\AppData\Local\Temp\ipykernel_20208\2259806315.py:130: RuntimeWarning:

invalid value encountered in scalar multiply



KeyboardInterrupt: 